# Apartado 1 Naive-Bayes propio
• Tabla con los resultados de la ejecución para los conjuntos de datos
analizados (wdbc y heart). Considerar los dos tipos de particionado. Los
resultados se refieren a las tasas de error y deben mostrarse tanto con
la corrección de Laplace como sin ella. Se debe incluir tanto el
promedio de error como su desviación típica. Es importante mostrar
todos los resultados agrupados en una tabla para facilitar su evaluación.

In [1]:
import pandas as pd
import numpy as np
import random
import EstrategiaParticionado
from Datos import Datos
from Clasificador import ClasificadorNaiveBayes
from os import listdir

resultados = []
# Damos un valor aleatorio a la semilla para cada ejecución
seed = random.random()

# Ejecutamos cada dataset que se encuentre en la carpeta Datasets
for archivo in listdir('Datasets/'):
    dataset = Datos('Datasets/' + archivo)
    
    # Parámetros de las estrategias de particionado
    n_ejecuciones = 5
    n_folds = 5
    estrategia_simple = EstrategiaParticionado.ValidacionSimple(n_ejecuciones, 0.25)
    estrategia_cruzada = EstrategiaParticionado.ValidacionCruzada(n_folds)
    clasificador = ClasificadorNaiveBayes(laplace=1)

    # Con corrección de Laplace
    error_simple_laplace = clasificador.validacion(estrategia_simple, dataset, clasificador, seed)
    error_cruzada_laplace = clasificador.validacion(estrategia_cruzada, dataset, clasificador)
    
    # Sin corrección de Laplace
    clasificador.laplace = 0
    error_simple = clasificador.validacion(estrategia_simple, dataset, clasificador, seed)
    error_cruzada = clasificador.validacion(estrategia_cruzada, dataset, clasificador)
    
    # Calculamos los promedios y las desviaciones típicas y las almacenamos en los resutlados 
    # para posteriormente mostrarlos en una tabla
    resultados.append({
        'Dataset': archivo,
        'Particionado': 'Simple',
        'Laplace': True,
        'Error Promedio': np.mean(error_simple_laplace),
        'Desviación Típica': np.std(error_simple_laplace)
    })
    resultados.append({
        'Dataset': archivo,
        'Particionado': 'Simple',
        'Laplace': False,
        'Error Promedio': np.mean(error_simple),
        'Desviación Típica': np.std(error_simple)
    })
    resultados.append({
        'Dataset': archivo,
        'Particionado': 'Cruzada',
        'Laplace': True,
        'Error Promedio': np.mean(error_cruzada_laplace),
        'Desviación Típica': np.std(error_cruzada_laplace)
    })
    resultados.append({
        'Dataset': archivo,
        'Particionado': 'Cruzada',
        'Laplace': False,
        'Error Promedio': np.mean(error_cruzada),
        'Desviación Típica': np.std(error_cruzada)
    })

# Convertimos los resultados en un dataframe
df_resultados = pd.DataFrame(resultados)

# Mostramos la tabla
display(df_resultados)


,Dataset,Particionado,Laplace,Error Promedio,Desviación Típica
0,heart.csv,Simple,True,0.145217,0.025589
1,heart.csv,Simple,False,0.145217,0.025589
2,heart.csv,Cruzada,True,0.146110,0.053510
3,heart.csv,Cruzada,False,0.146110,0.053510
4,wdbc.csv,Simple,True,0.054930,0.022001
5,wdbc.csv,Simple,False,0.054930,0.022001
6,wdbc.csv,Cruzada,True,0.070315,0.031910
7,wdbc.csv,Cruzada,False,0.070315,0.031910


• Breve análisis de los resultados anteriores. Discutir el efecto Laplace.
Para la creación de particiones se han empleado parámetros estándar que son los que se encuentran por defecto en las versiones correspondientes de scikit learn, en concreto,
para la validación simple se ha utilizado un porcentaje del 75% (1 - 0.25) para el tamaño de la partición de entrenamiento y para la validación cruzada se han utilizado 5 folds.
Atendiendo a los resultados del conjunto de datos "wdbc.csv" podemos ver que tanto el error promedio como la desviación típica son muy bajos, indicando que la clasificación es casi perfecta teniendo una tasa de error promedio cercana al 5-7%, esto puede deberse a que los datos continuos siguen realmente una distribución Gaussiana y por lo tanto al suponer nuestro Naive Bayes una distribución Gaussiana para estos datos se obtiene este error tan bajo. Por otro lado, observamos que la corrección Laplaciana no afecta al resultado, esto se explica por el hecho de que el conjunto de datos unicamente cuenta con atributos continuos, y no aplicamos dicha corrección para atributos continuos. 
Analizando los resultados del conjunto "heart.csv" observamos unos ratios de error ligeramente más elevados que para el otro conjunto, esto posiblemente se pueda explicar por el hecho de que alguno de los atributos no siga una distribución Gaussiana, sin embargo, el error sigue siendo relativamente bajo rondando un 15%. En cuanto a Laplace para conjuntos de entrenamiento grandes no se observa ninguna diferencia significativa puesto que no hay datos que falten en el entrenamiento y Laplace no tiene efecto, sin embargo, si reducimos significativamente el tamaño del conjunto de entrenamiento podemos ver una diferencia alrededor del 2% entre aplicar el suavizado o no, siendo el NB con suavizado más preciso.

# Apartado 2 Naive-Bayes Scikit-Learn
• Tabla de resultados equivalente a la anterior, pero utilizando los
métodos del paquete scikit-learn: MultinomialNB, GaussianNB y
CategoricalNB.

In [1]:
import pandas as pd
import numpy as np
from os import listdir
from sklearn import model_selection
from sklearn import naive_bayes as nb
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors as knn

def validacion_simple(datos_categoricos, datos_numericos, nb_gaussian, nb_categorical,
           test_num, test_cat, test_y):
    if datos_categoricos is not None and datos_numericos is not None:
        error_num = 1 - nb_gaussian.score(test_num, test_y)
        error_cat = 1 - nb_categorical.score(test_cat, test_y)

        peso_num = datos_numericos.shape[1] / (datos_numericos.shape[1] + datos_categoricos.shape[1])
        peso_cat = datos_categoricos.shape[1] / (datos_numericos.shape[1] + datos_categoricos.shape[1])

        error_promedio = (error_num * peso_num) + (error_cat * peso_cat)
    elif datos_categoricos is None and datos_numericos is not None:
        error_num = 1 - nb_gaussian.score(test_num, test_y)
        error_promedio = error_num
    elif datos_categoricos is not None and datos_numericos is None:
        error_cat = 1 - nb_categorical.score(test_cat, test_y)
        error_promedio = error_cat
    else:
        error_promedio = -1

    return error_promedio

def validacion_cruzada(datos_categoricos_codificados, datos_numericos, nb_gaussian, nb_categorical,
                      target): 
    if datos_categoricos_codificados is not None and datos_numericos is not None:
        error_num = 1 - \
            model_selection.cross_val_score(
                nb_gaussian, datos_numericos, target, cv=5)
        error_cat = 1 - \
            model_selection.cross_val_score(
                nb_categorical, datos_categoricos_codificados, target, cv=5)
        error_promedio = (error_num + error_cat) / 2
    elif datos_categoricos_codificados is None and datos_numericos is not None:
        error_num = 1 - \
            model_selection.cross_val_score(
                nb_gaussian, datos_numericos, target, cv=5)
        error_promedio = error_num
    elif datos_categoricos_codificados is not None and datos_numericos is None:
        error_cat = 1 - \
            model_selection.cross_val_score(
                nb_categorical, datos_categoricos_codificados, target, cv=5)
        error_promedio = error_cat
    else:
        error_promedio = -1

    return error_promedio

resultados = []

for archivo in listdir('Datasets/'):
    dataset = 'Datasets/' + archivo
    df = pd.read_csv(dataset, dtype={'Class': 'object'})

    # Separamos características numéricas y categóricas
    datos_numericos = df.select_dtypes(include='number')
    datos_categoricos = df.select_dtypes(include='object')

    # Separamos la columna target
    target = df['Class']
    datos_categoricos = datos_categoricos.drop('Class', axis=1)

    # Codificación One-Hot para datos categóricos si existen
    datos_categoricos_codificados = None
    if not datos_categoricos.empty:
        datos_categoricos_codificados = pd.get_dummies(
            datos_categoricos, drop_first=True)
    else:
        datos_categoricos = None

    # Si no hay datos numericos None
    if datos_numericos.empty:
        datos_numericos = None

    # Concatenamos los datos numéricos y categóricos codificados
    X = pd.concat([datos_numericos, datos_categoricos_codificados], axis=1)

    n_ejecuciones = 5
    lista_errores_simple_laplace_cat = []
    lista_errores_simple_cat = []
    lista_errores_simple_mult = []

    nb_gaussian = nb.GaussianNB()
    nb_categorical_laplace = nb.CategoricalNB()
    nb_categorical = nb.CategoricalNB(alpha=0)
    nb_multinomial = nb.MultinomialNB()
    # Realizamos la división de los datos
    for i in range(n_ejecuciones):
        train_X, test_X, train_y, test_y = model_selection.train_test_split(
            X, target, test_size=0.25)
    
        # Ahora separamos los datos numéricos y categóricos a partir de los conjuntos de entrenamiento y prueba
        train_num = None
        test_num = None
        if datos_numericos is not None:
            train_num = train_X[datos_numericos.columns]
            test_num = test_X[datos_numericos.columns]
    
        train_cat = None
        test_cat = None
        if datos_categoricos_codificados is not None:
            train_cat = train_X[datos_categoricos_codificados.columns]
            test_cat = test_X[datos_categoricos_codificados.columns]
    
        # Naive Bayes para atributos numéricos
        if datos_numericos is not None:
            nb_gaussian.fit(train_num, train_y)
    
        # Naive Bayes para atributos categóricos con corrección de Laplace y sin ella
        if datos_categoricos is not None:
            nb_categorical_laplace.fit(train_cat, train_y)
            nb_categorical.fit(train_cat, train_y)

        # Naive Bayes multinomial
        if datos_categoricos is not None:
            nb_multinomial.fit(train_cat, train_y)
    
        # ValidaciónSimple
        lista_errores_simple_laplace_cat.append(validacion_simple(datos_categoricos, datos_numericos, \
                                                 nb_gaussian, nb_categorical_laplace, test_num, test_cat, test_y))
        lista_errores_simple_cat.append(validacion_simple(datos_categoricos, datos_numericos, \
                                         nb_gaussian, nb_categorical, test_num, test_cat, test_y))
        lista_errores_simple_mult.append(validacion_simple(datos_categoricos, datos_numericos, \
                                         nb_gaussian, nb_multinomial, test_num, test_cat, test_y))

    # ValidaciónCruzada
    error_cruzada_laplace_cat = validacion_cruzada(datos_categoricos_codificados, datos_numericos, nb_gaussian, \
                                              nb_categorical_laplace, target)
    error_cruzada_cat = validacion_cruzada(datos_categoricos_codificados, datos_numericos, nb_gaussian, \
                                              nb_categorical, target)
    error_cruzada_mult = validacion_cruzada(datos_categoricos_codificados, datos_numericos, nb_gaussian, \
                                              nb_multinomial, target)

    # Calculamos los promedios y las desviaciones típicas y las almacenamos en los resutlados 
    # para posteriormente mostrarlos en una tabla
    resultados.append({
        'Dataset': archivo,
        'Particionado': 'Simple',
        'NB': 'Gaussian y Categorical',
        'Laplace': True,
        'Error Promedio': np.mean(lista_errores_simple_laplace_cat),
        'Desviación Típica': np.std(lista_errores_simple_laplace_cat)
    })
    resultados.append({
        'Dataset': archivo,
        'Particionado': 'Simple',
        'NB': 'Gaussian y Categorical',
        'Laplace': False,
        'Error Promedio': np.mean(lista_errores_simple_cat),
        'Desviación Típica': np.std(lista_errores_simple_cat)
    })
    resultados.append({
        'Dataset': archivo,
        'Particionado': 'Cruzada',
        'NB': 'Gaussian y Categorical',
        'Laplace': True,
        'Error Promedio': np.mean(error_cruzada_laplace_cat),
        'Desviación Típica': np.std(error_cruzada_laplace_cat)
    })
    resultados.append({
        'Dataset': archivo,
        'Particionado': 'Cruzada',
        'NB': 'Gaussian y Categorical',
        'Laplace': False,
        'Error Promedio': np.mean(error_cruzada_cat),
        'Desviación Típica': np.std(error_cruzada_cat)
    })
    
    resultados.append({
        'Dataset': archivo,
        'Particionado': 'Simple',
        'NB': 'Gaussian y Multinomial',
        'Error Promedio': np.mean(lista_errores_simple_mult),
        'Desviación Típica': np.std(lista_errores_simple_mult)
    })
    resultados.append({
        'Dataset': archivo,
        'Particionado': 'Cruzada',
        'NB': 'Gaussian y Multinomial',
        'Error Promedio': np.mean(error_cruzada_mult),
        'Desviación Típica': np.std(error_cruzada_mult)
    })

# Convertimos los resultados en un dataframe
df_resultados = pd.DataFrame(resultados)

# Mostramos la tabla
display(df_resultados)

/home/santacg/Code/Venvs/FAA_venv/lib/python3.12/site-packages/sklearn/naive_bayes.py:1504: RuntimeWarning: divide by zero encountered in log
  np.log(smoothed_cat_count) - np.log(smoothed_class_count.reshape(-1, 1))
/home/santacg/Code/Venvs/FAA_venv/lib/python3.12/site-packages/sklearn/naive_bayes.py:1504: RuntimeWarning: divide by zero encountered in log
  np.log(smoothed_cat_count) - np.log(smoothed_class_count.reshape(-1, 1))
/home/santacg/Code/Venvs/FAA_venv/lib/python3.12/site-packages/sklearn/naive_bayes.py:1504: RuntimeWarning: divide by zero encountered in log
  np.log(smoothed_cat_count) - np.log(smoothed_class_count.reshape(-1, 1))


,Dataset,Particionado,NB,Laplace,Error Promedio,Desviación Típica
0,wdbc.csv,Simple,Gaussian y Categorical,True,0.065065,0.007877
1,wdbc.csv,Simple,Gaussian y Categorical,False,0.065065,0.007877
2,wdbc.csv,Cruzada,Gaussian y Categorical,True,0.061481,0.014586
3,wdbc.csv,Cruzada,Gaussian y Categorical,False,0.061481,0.014586
4,wdbc.csv,Simple,Gaussian y Multinomial,NaN,0.065065,0.007877
5,wdbc.csv,Cruzada,Gaussian y Multinomial,NaN,0.061481,0.014586
6,heart.csv,Simple,Gaussian y Categorical,True,0.212225,0.013178
7,heart.csv,Simple,Gaussian y Categorical,False,0.215141,0.017723
8,heart.csv,Cruzada,Gaussian y Categorical,True,0.212530,0.050886
9,heart.csv,Cruzada,Gaussian y Categorical,False,0.213073,0.050688


• ¿Existe algún problema con alguno de estos métodos en alguno de los
dos ficheros? En caso afirmativo, ¿en cuál? ¿por qué? ¿cómo podría
resolverse?
En "wdbc.csv" tanto MultinomialNB como CategoricalNB presentan problemas porque el conjunto de datos contiene solo valores numéricos continuos, y estos modelos están diseñados para tratar unicamente con datos categóricos, con lo cual, la solución adecuada para "wdbc.csv" pasa por usar GaussianNB, el cual está diseñado para datos numéricos continuos, o discretizar las variables continuas, en este caso hemos optado por emplear GaussianNB si no existen datos categóricos, como es el caso de dicho conjunto. En "heart.csv" hay datos mixtos, es decir, numéricos y categóricos, por ende MultinomialNB y CategoricalNB tendrán problemas para los atributos continuos y GaussianNB para los atributos discretos o categóricos, la solución por la que se ha optado es un modelo híbrido que emplee GaussianNB para los datos continuos y CategoricalNB o MultinomialNB para los datos categóricos (se toma la media ponderada por el número de atributos categóricos y continuos del error de ambos modelos para ello)

# Apartado 3 K-NN propio
Resultados en forma de tabla de la clasificación mediante
vecinos próximos para los diferentes valores de vecindad en
los conjuntos de datos propuestos. Obtener los resultados
tanto para datos estandarizados como sin estandarizar, con el
objetivo de justificar el rendimiento del algoritmo en base a
estas características. Separar por tipo de validación (simple,
cruzada)

In [ ]:
import numpy as np
import pandas as pd
from Datos import Datos
import EstrategiaParticionado
from Clasificador import ClasificadorKNN
from os import listdir

# Datasets a utilizar
datasets = ['heart.csv', 'wdbc.csv']

# Valores de K a probar
K_values = [1, 5, 11, 21]

normalizations = [False, True]

# Número de ejecuciones y folds
n_ejecuciones = 5
n_folds = 5

resultados = []

for archivo in datasets:
    dataset = Datos('Datasets/' + archivo)

    for normalizado in normalizations:
        if normalizado:
            normalizacion = 'Normalizado'
        else:
            normalizacion = 'No Normalizado'

        estrategia_simple = EstrategiaParticionado.ValidacionSimple(
            n_ejecuciones, 0.25)
        estrategia_cruzada = EstrategiaParticionado.ValidacionCruzada(n_folds)

        for K in K_values:
            # Validación Simple
            clasificador_simple = ClasificadorKNN(
                K=K, normalize=normalizado)
            errores_simple = clasificador_simple.validacion(
                estrategia_simple, dataset, clasificador_simple)

            # Validación Cruzada
            clasificador_cruzada = ClasificadorKNN(K=K, normalize=normalizado)
            errores_cruzada = clasificador_cruzada.validacion(
                estrategia_cruzada, dataset, clasificador_cruzada)

            resultados.append({
                'Dataset': archivo,
                'Particionado': 'Simple',
                'K': K,
                'Normalizado': normalizacion,
                'Error Promedio': np.mean(errores_simple),
                'Desviación Típica': np.std(errores_simple)
            })
            resultados.append({
                'Dataset': archivo,
                'Particionado': 'Cruzada',
                'K': K,
                'Normalizado': normalizacion,
                'Error Promedio': np.mean(errores_cruzada),
                'Desviación Típica': np.std(errores_cruzada)
            })

df_resultados = pd.DataFrame(resultados)

print(df_resultados)